<a href="https://colab.research.google.com/github/ReidelVichot/LC_identification/blob/main/EmpByInd_011725.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. PROBLEM DEFINITION

**Background**

“A logistics cluster (LC) is defined as the geographical concentration of firms providing logistics services, such as transportation carriers, warehousing providers, third-party logistics (3PL-s), and forwarders, as well as some other enterprises that are mainly in the logistics business, including logistics enterprises to provide services to various industries” (Rivera et al., 2014, p. 223).  

Several relevant scholars in the field of logistics claim that clustering logistic activity has a positive impact on the efficiency of the economic activity, reduction of costs, and increase of collaboration among the firms that belong to the cluster (Rivera et al., 2014; Rivera, Gligor, et al., 2016; Rivera, Sheffi, et al., 2016; Sheffi, 2013, 2012). Although some of these authors mention that some of these benefits require some trade-offs (Rivera, Gligor, et al., 2016), these trade-offs are not further explored, resulting in an incomplete understanding of the socio-economic effects of the agglomeration of logistics activity. This becomes more problematic given that governments around the world seem to be embracing the idea of logistics clusters being some sort of panacea for economic development based on supply chain management improvements (Baranowski et al., 2015; Baydar et al., 2019; Chung, 2016), even though empirical studies that assess the role of government spending on the formation of logistics clusters are lacking (Liu et al., 2022). In other words, the field still lacks methodological and theoretical development, resulting in an incomplete understanding of the mechanisms of logistical clustering and their socio-economic effects.

**Problem**

There is not a current database of logistics clusters in the US. However, Rivera et al (2014) designed a method to test logistical agglomeration in US counties using NAICS codes and [CBP](https://www.census.gov/programs-surveys/cbp.html) information. Before conducting analyis on the effects of Logistics Clusters on society and the role of governments in their formation it is necessary to have an accurate picture of all logistics clusters in the US. For this purpose, I will extend Reviera's et al (2014) methodology to all the CBP years in which NAICS codes are used and use this database for future analyses.

#2. DATA COLLECTION

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import time

# -- this line is to make pandas future-proof, Copy-on-Write will become the default in pandas 3.0.
pd.options.mode.copy_on_write = True

pd.set_option("future.no_silent_downcasting", True)

# -- Set the data path
dpath = "/content/drive/MyDrive/Disertation/"

In [ ]:
t0 = time.time()
for year in range(1998, 2022):
  xx = str(year)[2:]
  fname = dpath + "CBP_data/cbp" + xx + "co/cbp" + xx + "co.txt"
  temp = pd.read_csv(fname)
  if year == 2015:
    temp.columns = temp.columns.str.lower()
  # -- add a year column
  temp["year"] = year
  # -- add a GEOID
  temp["GEOID"] = temp.fipstate.astype(str).str.zfill(2) + temp.fipscty.astype(str).str.zfill(3)
  # -- create a global variable and save a dataframe into it
  # -- temp1 contains the 2-digits naics codes
  temp1 = temp[temp["naics"].str.endswith("----")]
  temp1 = temp1[~temp1["naics"].str.startswith("--")]
  temp1["naics"] = temp1["naics"].str[:2]
  # -- get the unique 2-digits naics codes
  digits = tuple(temp1["naics"].unique())

  # -- get 3-digit naics that do not include the 2-digit ones.
  temp2 =temp[temp['naics'].str.endswith('///')]
  temp2 = temp2[~temp2['naics'].str.startswith(digits)]
  temp2['naics'] = temp2['naics'].str[:2]


  globals()["ind" + xx] = pd.concat([temp1, temp2])
  # -- delete to save RAM
  del temp
  del temp1
  del temp2
t1 = time.time()
print("Execution Time: ", (t1 - t0)/60, " mins")

Execution Time:  3.3962438980738323  mins


#3. DATA PREPARATION

In [ ]:
# The CBP record layouts are different accross years.
cols_98_16 = ['fipstate', 'fipscty', 'naics', 'empflag', 'emp', 'ap', 'est',
              'n1_4', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499',
              'n500_999','n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4',
              'year', 'GEOID']

cols_17    = ['fipstate', 'fipscty', 'naics', 'empflag', 'emp', 'ap', 'est',
              'n<5', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499',
              'n500_999','n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4',
              'year', 'GEOID']

cols_18_21 = ['fipstate', 'fipscty', 'naics', 'emp_nf',  'emp', 'ap', 'est',
              'n<5', 'n5_9', 'n10_19', 'n20_49', 'n50_99', 'n100_249', 'n250_499',
              'n500_999','n1000', 'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4',
              'year', 'GEOID']

t0 = time.time()
for year in range(1998, 2017):
  xx = str(year)[2:]
  globals()["ind" + xx] = globals()["ind" + xx][cols_98_16]

ind17 = ind17[cols_17]
ind17.columns = ind98.columns

#####  NOTE: CBP17 has object-type columns for the employment by establishment size.
# from 2017-2021 there are values that have N instead of 0 for emp by est size
ind17[ind17.columns[7:19]] = ind17[ind17.columns[7:19]].replace("N", 0).astype(int)

for year in range(2018, 2022):
  xx = str(year)[2:]
  globals()["ind" + xx] = globals()["ind" + xx][cols_18_21]
  globals()["ind" + xx][globals()["ind" + xx].columns[7:19]] = globals()["ind" + xx][globals()["ind" + xx].columns[7:19]].replace("N", 0).astype(int)
  globals()["ind" + xx].columns = ind98.columns

# -- Between 1998 and 2017, there was a suppression flag (empflag) that affects
#    the employment. Instead of using employment as it is, I will adjust it
#    based on the middle point of the employment by establishment size.

# -- create an unified dataframe for all years (1998-2017)
frames = [ind98, ind99, ind00, ind01, ind02, ind03, ind04, ind05, ind06, ind07,
          ind08, ind09, ind10, ind11, ind12, ind13, ind14, ind15, ind16, ind17,
          ind18, ind19, ind20, ind21]

ind = pd.concat(frames).reset_index().drop(columns="index")
# -- lambda funditon to estimate employment if the employment is flagged
t0 = time.time()
ind['emp_adj'] = ind.apply(lambda row: row["emp"]
                           if pd.isna(row['empflag'])
                           else row['n1_4']*2.5 + row['n5_9']*7 +
                                row['n10_19']*14.5 + row['n20_49']*34.5 +
                                row['n50_99']*74.5 + row['n100_249']*174.5 +
                                row['n250_499']*374.5 + row['n500_999']*749.5 +
                                row['n1000']*1000 , axis=1)
ind = ind.drop(columns=['n1_4', 'n5_9', 'n10_19', 'n20_49', 'n50_99',
                        'n100_249', 'n250_499', 'n500_999', 'n1000',
                        'n1000_1', 'n1000_2', 'n1000_3', 'n1000_4'])
ind["emp_adj"] = ind.apply(lambda row: row["emp"] if (row["year"]==2018)|(row["year"]==2019)|(row["year"]==2020)|(row["year"]==2021) else row["emp_adj"], axis=1)

# this cbp is from 1998-2021
ind = ind.groupby(["GEOID", "year", "fipstate", "fipscty", "naics"]).sum(numeric_only=True).reset_index()

# -- save the cvs to use it later
ind.to_csv(dpath + "industries.csv", index = False)

t1 = time.time()
print("Execution Time: ", (t1-t0)/60, " mins")

Execution Time:  1.7032416184743246  mins


In [58]:
# Run this line to get access to the most current version of the industries
ind_temp = pd.read_csv(dpath + "industries.csv")
ind_temp["GEOID"] = ind_temp["GEOID"].astype(str).apply(lambda x: x.zfill(5))
ind = ind_temp
del ind_temp

In [59]:
# --Removing non-contiguous US counties
# Alaska, Hawaii, American Samoa, Guam, Nothern Marianas, Puerto Rico,
# Virgin Islands
nc_states = [2, 15, 60, 66, 69, 72, 78]
for i in nc_states:
  print("IND      ", len(ind))
  ind = ind[ind["fipstate"] != i]

IND       1624349
IND       1611442
IND       1608963
IND       1608963
IND       1608963
IND       1608963
IND       1608963


cat_ind=1 //Agriculture, forestry, fisheries, mining, and construction

cat_ind=2  //Manufacturing

cat_ind=3  //Transportation, communications, utilities, wholesale, and retail trade

cat_ind=4  //Finance, insurance, real estate, and business, repair, and personal services

cat_ind=5  //Entertainment and recreation, professional and related services, public administration, and active duty military


In [60]:
ind['naics'] = ind['naics'].astype(str).str[:2].astype(int)
# Function to categorize industries based on NAICS codes
def categorize_industry(naics_code):
    if naics_code in [11, 21, 23]:  # Agriculture, Forestry, Fishing, Mining, Construction
        return 1
    elif 31 <= naics_code <= 33:  # Manufacturing
        return 2
    elif naics_code in [42, 44, 45, 48, 49, 22, 51]:  # Transportation, Communications, Utilities, Wholesale, Retail
        return 3
    elif naics_code in [52, 53, 54, 55, 56, 81]:  # Finance, Insurance, Real Estate, Business Services
        return 4
    elif naics_code in [51, 61, 62, 71, 72, 81, 92, 95, 99]:  # Other Services, Public Administration
        return 5
    else:
        return None  # Handle cases with invalid or uncategorized NAICS codes

# Apply the function to create the 'cat_ind' variable
ind['cat_ind'] = ind['naics'].apply(categorize_industry)

# Grouping industries before classifying them
ind = ind.groupby(["GEOID","year", "cat_ind"]).sum().reset_index()
ind = ind.drop(columns=["fipstate", "fipscty", "naics"]).copy()

In [61]:
# Making categories a dummy variable
ind["ind_1"] = np.where(ind['cat_ind'] == 1, 1, 0)
ind["ind_2"] = np.where(ind['cat_ind'] == 2, 1, 0)
ind["ind_3"] = np.where(ind['cat_ind'] == 3, 1, 0)
ind["ind_4"] = np.where(ind['cat_ind'] == 4, 1, 0)
ind["ind_5"] = np.where(ind['cat_ind'] == 5, 1, 0)

# Organizing categories in the same line
ind["ind_1"] = ind["ind_1"]*ind["emp_adj"]
ind["ind_2"] = ind["ind_2"]*ind["emp_adj"]
ind["ind_3"] = ind["ind_3"]*ind["emp_adj"]
ind["ind_4"] = ind["ind_4"]*ind["emp_adj"]
ind["ind_5"] = ind["ind_5"]*ind["emp_adj"]

ind = ind.groupby(["GEOID", "year"]).sum().reset_index()

# Transforming to percentage
ind["ind_1_%"] = ind["ind_1"]/ind["emp_adj"]
ind["ind_2_%"] = ind["ind_2"]/ind["emp_adj"]
ind["ind_3_%"] = ind["ind_3"]/ind["emp_adj"]
ind["ind_4_%"] = ind["ind_4"]/ind["emp_adj"]
ind["ind_5_%"] = ind["ind_5"]/ind["emp_adj"]

# Dropping categories
ind.drop(columns=["cat_ind", "emp", "ap", "est"], inplace=True)

In [62]:
ind.describe()

,year,emp_adj,ind_1,ind_2,ind_3,ind_4,ind_5,ind_1_%,ind_2_%,ind_3_%,ind_4_%,ind_5_%
count,75693.000000,7.569300e+04,75693.000000,7.569300e+04,7.569300e+04,7.569300e+04,7.569300e+04,75693.000000,75693.000000,75693.000000,75693.000000,75693.000000
mean,2009.497404,4.272706e+04,2238.382426,7.386786e+03,1.118968e+04,1.097083e+04,1.094139e+04,0.076137,0.224666,0.272597,0.163380,0.263221
std,6.920095,1.529228e+05,7441.503753,2.415600e+04,4.019179e+04,5.013977e+04,3.916999e+04,0.075861,0.162669,0.087095,0.104584,0.107272
min,1998.000000,2.500000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2004.000000,2.549500e+03,162.000000,3.620000e+02,6.340000e+02,3.190000e+02,6.155000e+02,0.036806,0.091735,0.221076,0.110426,0.195815
50%,2009.000000,7.934000e+03,459.000000,1.816000e+03,2.019500e+03,9.910000e+02,1.832000e+03,0.054739,0.201393,0.268007,0.142490,0.254642
75%,2015.000000,2.454600e+04,1405.000000,6.136000e+03,6.234000e+03,3.730000e+03,6.086000e+03,0.086207,0.331307,0.316877,0.186098,0.321579
max,2021.000000,4.463569e+06,211082.000000,1.110974e+06,1.311745e+06,1.523754e+06,1.328896e+06,1.000000,0.860821,1.000000,1.000000,1.000000


#4. MACHINE LEARNING

#5. PROBLEM SOLUTION